# 0. Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
import warnings

# Filter out the specific UserWarnings
warnings.filterwarnings("ignore", category=UserWarning, message="A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy")
warnings.filterwarnings("ignore", category=UserWarning, message="unable to load libtensorflow_io_plugins.so")
warnings.filterwarnings("ignore", category=UserWarning, message="file system plugins are not loaded")

In [ ]:
from transformers import AutoTokenizer, TFAutoModel

In [ ]:
from datasets import Dataset, DatasetDict

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import classification_report

In [ ]:
import tensorflow as tf

from tensorflow.keras.optimizers import AdamW

from tensorflow.keras.losses import SparseCategoricalCrossentropy

from tensorflow.keras.layers import Input, Dense, GlobalMaxPooling1D

from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler

# 1. Load Dataset

In [ ]:
def load_data(file_path):
    return pd.read_csv(file_path, header=None, delimiter='\t', names = ['emotion','text'])

train_path = '/kaggle/input/emotion/train-emotion-all.tsv'
test_path = '/kaggle/input/emotion/test-emotion-all.tsv'
val_path = '/kaggle/input/emotion/valid-emotion-all.tsv'

df_train = load_data(train_path)
df_test = load_data(test_path)
df_val = load_data(val_path)

In [ ]:
pd.set_option('display.max_colwidth', 150)
df_train.head()

## 1.2 Preparing datasets

In [ ]:
encoded_dict = {'TRISTEZZA':0, 'GIOIA':1, 'AMORE':2, 'RABBIA':3, 'PAURA':4, 'SORPRESA':5, 'NEUTRA':6}

df_train['label'] = df_train['emotion'].apply(lambda x: encoded_dict[x])
df_test['label'] = df_test['emotion'].apply(lambda x: encoded_dict[x])
df_val['label'] = df_val['emotion'].apply(lambda x: encoded_dict[x])

In [ ]:
def label2id(label):
    if isinstance(label, list):
        return [encoded_dict[label] for label in label]
    else:
        return encoded_dict[label]

def id2label(id):
    encoded_dict_inv = {v: k for k, v in encoded_dict.items()}
    
    if isinstance(id, list):
        return [encoded_dict_inv[i] for i in id]
    else:
        return encoded_dict_inv[id]

In [ ]:
train_dataset = Dataset.from_pandas(df_train)
test_dataset = Dataset.from_pandas(df_test)
val_dataset = Dataset.from_pandas(df_val)

# Create the DatasetDict
dataset = DatasetDict({'train': train_dataset, 'test': test_dataset, 'validation': val_dataset})

print(dataset)

In [ ]:
# Initialize a dictionary to store updated datasets
updated_datasets = {}

# Check for and remove duplicates in each split
for split in dataset.keys():
    split_data = dataset[split]
    
    # Access the 'text' column within the list
    text_column = split_data['text']
    
    # Initialize a set to track unique texts
    unique_texts = set()
    
    # Initialize lists to store the filtered data
    filtered_text = []
    
    # Iterate through the 'text' column and filter duplicates
    for text in text_column:
        if text not in unique_texts:
            unique_texts.add(text)
            filtered_text.append(text)
    
    # Create a new Dataset object with the filtered data
    updated_datasets[split] = split_data.select(list(range(len(filtered_text))))
    
    # Print the number of removed duplicates
    duplicate_count = len(text_column) - len(filtered_text)
    print(f"Duplicates removed in {split} split: {duplicate_count}\n")

# Update the dataset dictionary with the filtered datasets
dataset.update(updated_datasets)

# Print the updated dataset information
for split in dataset.keys():
    split_data = dataset[split]
    print(f"{split}: {len(split_data['text'])} rows")

print(dataset)

# 2. Preprocess data

In [ ]:
bert = TFAutoModel.from_pretrained('Twitter/twhin-bert-base', from_pt=True)
tokenizer = AutoTokenizer.from_pretrained('Twitter/twhin-bert-base')

In [ ]:
max_length = 256

def tokenize_text(dataset):
    return tokenizer(
        text=dataset['text'],
        add_special_tokens=True,
        return_token_type_ids=False,
        max_length=max_length,
        padding='max_length',
        truncation=True,
        return_tensors='tf',
        verbose=True
    )

In [ ]:
encoded_dataset = dataset.map(tokenize_text)
encoded_dataset

In [ ]:
encoded_dataset = encoded_dataset.remove_columns(['emotion','text'])

encoded_dataset

In [ ]:
def preprocess_data(encoded_dataset, data_type):
    input_ids = np.array(encoded_dataset[data_type]['input_ids'])
    input_ids = np.squeeze(input_ids, axis=1)

    attention_mask = np.array(encoded_dataset[data_type]['attention_mask'])
    attention_mask = np.squeeze(attention_mask, axis=1)

    label = np.array(encoded_dataset[data_type]['label'])

    return input_ids, attention_mask, label

def main_processing(encoded_dataset):
    input_ids_train, attention_mask_train, label_train = preprocess_data(encoded_dataset, 'train')
    input_ids_val, attention_mask_val, label_val = preprocess_data(encoded_dataset, 'validation')
    input_ids_test, attention_mask_test, label_test = preprocess_data(encoded_dataset, 'test')

    return input_ids_train, attention_mask_train, label_train, input_ids_val, attention_mask_val, label_val, input_ids_test, attention_mask_test, label_test

# Usage
input_ids_train, attention_mask_train, label_train, input_ids_val, attention_mask_val, label_val, input_ids_test, attention_mask_test, label_test = main_processing(encoded_dataset)

# 3. Defining the model

In [ ]:
input_ids = Input(shape=(max_length,), dtype=tf.int32, name="input_ids")
input_mask = Input(shape=(max_length,), dtype=tf.int32, name="attention_mask")

embeddings = bert.bert(input_ids, attention_mask = input_mask)[0]
out = GlobalMaxPooling1D(name="GlobalMaxPooling1d")(embeddings)
out = Dense(128, activation='relu',name="Dense_relu")(out)

y = Dense(7, activation='softmax')(out)
    
model = tf.keras.Model(inputs=[input_ids, input_mask], outputs=y)
model.layers[2].trainable = True

In [ ]:
optimizer = AdamW(
    learning_rate=1e-5,
    epsilon=1e-08,
    weight_decay=0.01,
    name="AdamW"
)

In [ ]:
for i, layer in enumerate(model.layers):
    print(f"Layer {i}: {layer.name}")

In [ ]:
loss = SparseCategoricalCrossentropy(
    from_logits=False,
    ignore_class=None,
    reduction="auto",
    name="sparse_categorical_crossentropy",
)

In [ ]:
def scheduler(epoch,lr):
    if epoch <2:
        return lr
    else:
        return lr*tf.math.exp(-0.1)
    
lr_scheduler = LearningRateScheduler(scheduler)

In [ ]:
early_stop = EarlyStopping(
    monitor="val_loss",
    min_delta=0,
    patience=4,
    verbose=0,
    mode="auto",
    baseline=None,
    restore_best_weights=True,
    start_from_epoch=0,
)

In [ ]:
model.compile(
    optimizer=optimizer,
    loss=loss,
    metrics=["sparse_categorical_accuracy"]
)

In [ ]:
model.summary()

# 4. Training

In [ ]:
history = model.fit(
    x = {'input_ids':input_ids_train, 'attention_mask':attention_mask_train},
    y = label_train,
    validation_data = ({'input_ids':input_ids_val, 'attention_mask':attention_mask_val},
                      (label_val)),
    epochs=10,
    batch_size=16,
    callbacks=[lr_scheduler, early_stop]
)

In [ ]:
plt.plot(history.history['loss'], label='train_loss')
plt.plot(history.history['val_loss'], label='val_loss')
plt.legend()
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.show()

# 5. Metrics

In [ ]:
predicted = model.predict({'input_ids': input_ids_test, 'attention_mask': attention_mask_test})
predicted_labels = np.argmax(predicted, axis=1)

In [ ]:
predicted_labels = predicted_labels.tolist()
predicted_labels = id2label(predicted_labels)
predicted_labels[0:7]

In [ ]:
label_test = label_test.tolist()
label_test = id2label(label_test)
label_test[0:7]

In [ ]:
print(classification_report(label_test, predicted_labels))

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns

# Compute the confusion matrix
conf_matrix = confusion_matrix(label_test, predicted_labels, labels =['NEUTRA', 'GIOIA', 'RABBIA', 'AMORE', 'PAURA', 'TRISTEZZA', 'SORPRESA'])

# Create a heatmap for the confusion matrix
plt.figure(figsize=(8, 6))

sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues',
            xticklabels=['NEUTRA', 'GIOIA', 'RABBIA', 'AMORE', 'PAURA', 'TRISTEZZA', 'SORPRESA'], 
            yticklabels=['NEUTRA', 'GIOIA', 'RABBIA', 'AMORE', 'PAURA', 'TRISTEZZA', 'SORPRESA'])

plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()


## 5.1 Accuracy Score

In [ ]:
accuracy = accuracy_score(label_test, predicted_labels) # (TP+TN)/P+N i.e total number of corrected classified tweet over total number of tweets

print(accuracy)

## 5.2 Precision Score

In [ ]:
precision = precision_score(label_test, predicted_labels,average=None, labels=['TRISTEZZA','GIOIA','AMORE','RABBIA','PAURA','SORPRESA','NEUTRA']) # TP/(TP+FP) i.e if predicted a certain class, which is the probability of being really that class?

print(precision)

## 5.3 Recall (sensitivity) Score

In [ ]:
recall = recall_score(label_test, predicted_labels,average=None, labels=['TRISTEZZA','GIOIA','AMORE','RABBIA','PAURA','SORPRESA','NEUTRA']) # TP/(TP+FN) i.e the ability of the estimator to predict all the tweets of a given class

print(recall)

## 5.4 F1 Score

In [ ]:
f1score = f1_score(label_test, predicted_labels,average=None, labels=['TRISTEZZA','GIOIA','AMORE','RABBIA','PAURA','SORPRESA','NEUTRA']) # 2*(precision*recall)/(precision+recall)

print(f1score)

# 6. To Hub

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from huggingface_hub import push_to_hub_keras

push_to_hub_keras(model, 'FedeBerto/Griffith-Emotion')

In [ ]:
from huggingface_hub import from_pretrained_keras

model = from_pretrained_keras('FedeBerto/Griffith-Emotion')